# Spacy
## Installazione e primi passi

Per installare **spacy** come prima cosa vanno installate le librerie *setuptools* e *wheel* perchè la libreria è disponibile come binary package.

```
pip install -U setuptools wheel
pip install -U spacy
```

Al centro della libreria c'è un oggetto *pipeline* che solitamente viene inizializzato usando il nome **nlp**.

Nella prossima cella inizializziamo una pipeline vuota per l'inglese col comando `spacy.blank("en")`, con questa variabile possiamo eseguere diversi compiti per l'analisi del testo perchè contiene già diverse funzioni della pipeline e le regole per la lingua scelta.

> Per cambiare lingua, ad esempio tedesco o italiano basterà usare `spacy.blank("de")` o `spacy.blank("it")`

<br>

![spacy_pipeline](https://spacy.io/images/pipeline.svg)

<br>

Nel prossimo esempio eseguiamo una semplice tokenizzazione del testo. Un token può essere una parola o la punteggiatura nel testo, per estrarre un token dal testo basta dare la sua posizione. L'oggetto *Token* di spacy dà accesso a diverse informazioni sui token, ad esempio la forma testuale dello stesso.

<br>

![token](https://course.spacy.io/doc.png)

<br>

In [1]:
import spacy

nlp = spacy.blank("en")

doc = nlp("Hello world!")

for token in doc:
    print(token.text)

print('\n')

print(doc[0].text)

Hello
world
!


Hello


Uno *span* non è altro che una parte di documento che consiste in uno o più token. Per creare uno span basta utilizzare la classica notazione python per selezionare parti di una stringa.

In [2]:
span = doc[1:3]

print(span)

world!


Per vedere gli attributi in una frase si possono usare gli attributi di spacy, ad esempio:
* `is_alpha` indica i token caratteri alfabetici
* `is_punct` la punteggiatura
* `like_num` i numeri, ma non solo se scritti sotto forma di cifre, anche come parola

Questi sono chiamati ***attributi lessicali*** dipendono dal dizionario utilizzato e non dal contesto della frase

In [3]:
doc = nlp("It coasts $5 .")

for token in doc:
    print("index: ", token.i)
    print("text: ", token.text)
    print("alpha: ", token.is_alpha)
    print("punct: ", token.is_punct)
    print("num: ", token.like_num)
    print('\n')

index:  0
text:  It
alpha:  True
punct:  False
num:  False


index:  1
text:  coasts
alpha:  True
punct:  False
num:  False


index:  2
text:  $
alpha:  False
punct:  False
num:  False


index:  3
text:  5
alpha:  False
punct:  False
num:  True


index:  4
text:  .
alpha:  False
punct:  True
num:  False




## Pipeline addestrate

Una pipeline addestrata è un oggetto SpaCy è un modello che è capace di estrarre attributi da un contesto (POS, NER, ...), è stato già addestrato su dati etichettati e può essere raffinato (fine-tuned) utilizzando nuovi dati.

<br>

![pos-tags](https://miro.medium.com/max/940/1*m2qeNjOSiDZzTFhdHpORqw.png)

<br>

SpaCy ha diverse pipeline pre-addestrate che possono essere scaricate mediante il comando `spacy.load`, ad esempio `en_core_web_sm` è una pipeline small, in inglese addestrata su testi presi dal web. Quando vengono scaricate le pipeline si scaricano i pesi per fare inferenza, i metadati sulla pipeline, il vocabolario e il file di configurazione su come è stata addestrata la pipeline.

Il primo esempio che vedremo è POS-tags utilizzando un modello preaddestrato. In questo caso oltre al testo ci facciamo restituire l'attibuto *pos_*.

> `pos_` ci restituisce la forma testuale, `pos` senza underscore restituisce l'ID del tag. In SpaCy questa è la convenzione per nominare degli attributi.

In [4]:
nlp = spacy.load('en_core_web_sm')

doc = nlp("I hate pizza with pineapple")


for token in doc:
    print(token.text, token.pos, token.pos_)

I 95 PRON
hate 100 VERB
pizza 92 NOUN
with 85 ADP
pineapple 92 NOUN


Oltre al POS-tags basico SpaCy restituisce anche le relazioni tra le parole, ad esempio se un nome è il soggetto o l'oggetto della frase o a chi/cosa si riferisce un articolo o un avverbio.

Nel nostro caso avremo:
* I: *nsubj* nominal subject
* pizza: *dobj* directed object
* pineapple: *pobj* object for preposition 

In [5]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head)

I PRON nsubj hate
hate VERB ROOT hate
pizza NOUN dobj hate
with ADP prep hate
pineapple NOUN pobj with


Prima di passare oltre andiamo a vedere cosa ha una pipeline preaddestrata sotto il cofano.

In [6]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Come possiamo vedere tra le componenti ci sono:
* **tok2vec**: per trasformare i token in vettori numerici
* **tagger**: per fare pos-tagging
* **parser**: per assegnare le dependency label
* **attribute_ruler**: mappa gli attributi dei token e assegna le eccezioni
* **lemmatizer**: indica i lemmi dei token
* **ner**: per fare name-entities-recognition

Nel prossimo esempio vedremo proprio la NER (Name Entities Recongnition). Mediante questa pipeline si assegna un'etichetta alle parole, ad esempio *persone*, *organizzazioni*, ...

La pipeline restituisce la NER tramite `.ents`, questo ritorna degli oggetti **Span** che a loro volta contengono testo ed etichette. Nel nostro esempio *Apple* *ORG* (organization) e *Cupertino* *GPE* (geopolitical entities).

> Nel secondo esempio abbiamo sostituito *Cupertino* con un più generico *Silicon Valley* notate che il tag cambia da *GPE*  a *LOC* (location). Possiamo capire perchè usando il comando `spacy.explain("GPE")` e `spacy.explain("LOC")`.

In [7]:
doc = nlp("Apple's headquarters is in Cupertino")

#doc = nlp("Apple's headquarters is in Silicon Valley")

#doc = nlp("Peter buys it on Amazon for 1$")

for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
Cupertino GPE


si possono visualizzare i risultati di una NER utilizzando la libreria *displacy*. Questa libreria ha due metodi in particolare `displacy.serve()` per avviare un web server e `displacy.render` per un markup.

In [8]:
from spacy import displacy

displacy.render(doc, style="ent")

con lo stesso comando ma con un cambio di stile possiamo vedere le dipendenze nella frase.

In [9]:
displacy.render(doc, style="dep")

## Rule-based match

Come scrivere delle regole per trovare parole e frasi nel testo non limitandosi alle espressioni regolari (*regex*).
Le regex si limitano a trovare stringhe, con questi oggetti invece possiamo cercare anche documenti e token, questo lo rende molto più flessibile.

I modelli di corrispondenza sono liste di dizionari, ogni dizionario è un token, la chiave è il nome del token `[{"TEXT": "iPhone"}, {"TEXT":"X"}]`. Si possono creare nuovi attributi mediante i token ad esempio `[{"LOWER":"iphone"}, {"LOWER":"x"}]`.

Per usarele corrispondenze (pattern) dobbiamo importare l'oggetto *Matcher* da spacy, creare un *oggetto nlp* caricando una pipeline e inizializzare il *Matcher* con il vocabolario della pipeline.

In [10]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

matcher = Matcher(nlp.vocab)

Il metodo `matcher.add` consente di aggiungere dei pattern, il primo elemento nel metodo è l'ID del pattern (una stringa), il secondo è il pattern (và passato come lista).

Per usare questo oggetto bisogna richiamarlo sul testo. Per visualizzare il risultato creiamo uno span utilizzando i seguenti elementi del matcher:
* match_id: hash value del pattern name
* start: indice iniziale del matched span
* end: indice finale del matched span

> il matcher ritorna liste di tuple, le tuple sono da tre elementi *[match_id, start, end]*

In [11]:
pattern = [{"TEXT":"iPhone"}, {"TEXT":"X"}]
matcher.add("IPHONE_PATTERN", [pattern])

doc = nlp("Peter buys an iPhone X on Amazon for 100$")

matches = matcher(doc)

for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

iPhone X


Vediamo come utilizzando i *Matcher* possiamo creare nuovi pattern, ad esempio il seguente basato sul verbo *love* (nelle sue declinazioni) seguito da un nome.

In [12]:
love_pattern = [
    {"LEMMA": "love", "POS": "VERB"},
    {"POS": "NOUN"}
]

matcher.add("LOVE_PATTERN", [love_pattern])

doc = nlp("I loved dogs but now I love cats more. Peter loves turtles")

love_matches = matcher(doc)

for match_id, start, end in love_matches:
    matched_span = doc[start:end]
    print(matched_span.text)

loved dogs
love cats
loves turtles


Nel prossimo esempio c'è un pattern che cerca un aggettivo seguito da un nome, e da un secondo nome che però è opzionale. Per creare questo pattern usiamo la combinazione ***"OP":"?"***.

> Per maggiori informazioni sul rule-based matching consultare la [documentazione](https://spacy.io/usage/rule-based-matching)

In [12]:
doc = nlp(
    "Features of the app include a beautiful design, smart search, automatic "
    "labels and optional voice responses."
)

pattern = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN", "OP": "?"}]

matcher.add("ADJ_NOUN_PATTERN", [pattern])
matches = matcher(doc)
print("Total matches found:", len(matches))

for match_id, start, end in matches:
    print("Match found:", doc[start:end].text)

Total matches found: 5
Match found: beautiful design
Match found: smart search
Match found: automatic labels
Match found: optional voice
Match found: optional voice responses
